In [ ]:
import numpy as np
import scipy as sc
import matplotlib.patches as patch
from scipy import stats
from matplotlib.legend_handler import HandlerLine2D
from skimage import data, io, color, transform, exposure
from pprint import pprint
from scipy import signal
from __future__ import print_function
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets

%matplotlib inline

import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = (32.0, 24.0)
pylab.rcParams['font.size'] = 24